# 非线性模型


In [2]:
import gurobipy as gp
import numpy as np
import math
import copy
import os
import sys

currentPath = os.getcwd().replace('\\','/')    # 获取当前路径

print(currentPath)


/Users/yurunfeng/Nutstore Files/我的坚果云/code/LR_in_python/My model


In [3]:
def previous(i, j, J):
    before = copy.deepcopy(J)
    before = [i] + before
    if j != 0:
        before.remove(j)
    return before


def later(i, j, bar_J):
    after = copy.deepcopy(bar_J)
    if i != j:
        after.remove(j)
    return after

def get_sol(x, I, bar_J, max_visit_num):
    plan = np.zeros((len(I), max_visit_num+1))
    for i in I:
        plan[i-I[0], 0] = i
        for k in later(i,i,bar_J):
            if math.isclose(x[i,i,k].X, 1, rel_tol=0.05):
                plan[i-I[0], 1] = k
                ind = k
                break
        
        count = 1
        flag = False
        while True:
            for k in later(i, ind, bar_J):
                if ind == bar_J[0]:
                    flag = True
                    break
                if math.isclose(x[i,ind,k].X, 1, rel_tol=0.05):
                    count = count+1
                    plan[i-I[0], count] = k
                    ind = k
            
            if flag:
                break
    return plan

## 导入数据

In [4]:
# 导入数据
np.set_printoptions(suppress=True)    # 取消numpy打印的科学计数法


# cost 矩阵的第一索引位置是0 默认为虚拟设施
root = '../data/SnyderData/15nodes/'
cost = np.loadtxt(root+'cost.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

dmd = np.loadtxt(root+'dmd.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

fc = np.loadtxt(root+'fc.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

## 参数设置

In [5]:
rho = 0.05 # 损坏概率参数
max_visit_num = 5 # 客户的最大尝试次数

q = rho * np.exp(-fc/200000) # 损坏概率
q[0] = 1

cus_num = len(dmd) # 客户数
node_num = cost.shape[0] # 节点数（虚拟 设施 客户）
fac_num = node_num - cus_num - 1 # 设施数

# 集合设置
J = [j for j in range(1, fac_num+1)] # 设施集合
I = [i for i in range(fac_num+1, node_num)] # 客户集合
bar_J = [j for j in range(0, fac_num+1)] # 设施拓展集合
R = [r for r in range(1,max_visit_num)] # 等级

# 常数集合
lmd = {i : dmd[i-fac_num-1] for i in I} # lambda需求
c = {(i,j) : cost[i,j] for i in bar_J+I for j in bar_J} # 价格
f = {j : fc[j] for j in J} # 建设成本


## 建模

In [6]:
m = gp.Model()

### 决策变量

In [7]:
y = m.addVars((j for j in J), vtype = gp.GRB.BINARY, name = 'y')
x = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), vtype = gp.GRB.BINARY,name = 'x') # 弧(j,j_p)属于客户i
p = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS,name = 'p') # 属于客户i的弧(j,j_p)的概率
w = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS,name = 'w') # 期望价格

## 目标函数
$$
\min C = \sum_{j\in J}f_jy_j + \sum_{i\in I}\lambda_i \sum_{k\in J_{ij}^+}\sum_{j\in \bar{J}} c_{kj} w_{ikj}
$$

In [8]:
item_1 = gp.quicksum(f[j] * y[j] for j in J)
item_2 = gp.quicksum((lmd[i] * c[k,j] * w[i,k,j]) for i in I for j in bar_J for k in previous(i,j,J))

m.setObjective(item_1 + item_2)
print("done")

done


## 约束
$$
% 指定给开启的节点
\sum_{k\in J_{ij}^+}x_{ikj} \le y_j ,\forall i \in I , j\in J\\
$$

In [9]:
m.addConstrs((gp.quicksum(x[i,k,j] for k in previous(i,j,J)) <= y[j] for i in I for j in J), name='assign2open')
print("done")

done


$$
% 流开始与结束
\sum_{j\in J_{ii}^-}x_{iij} = \sum_{j\in J_{i{j_{0}}}^+} x_{ijj_0} = 1, \forall i \in I \\
$$

In [10]:
m.addConstrs((gp.quicksum(x[i,i,j] for j in later(i,i,bar_J)) == 1 for i in I), name = 'flowin')
m.addConstrs((gp.quicksum(x[i,j,0] for j in previous(i,0,J)) == 1 for i in I), name = 'flowout')
print("done")

done


$$
% 流平衡
\sum_{k\in J_{ij}^-} x_{ijk} = \sum_{k\in J_{ij}^+} x_{ikj}, \forall i \in I, j \in J \\
$$

In [11]:
m.addConstrs((gp.quicksum(x[i,j,k] for k in later(i,j,bar_J)) 
           == gp.quicksum(x[i,k,j] for k in previous(i,j,J)) for i in I for j in J), name = 'balance')
print("done")

done


$$
% 概率初始化
p_{iij} = x_{iij}, \forall i\in I, j\in J_{ii}^- \\
$$

In [12]:
m.addConstrs((p[i,i,j] == x[i,i,j] for i in I for j in later(i,i,bar_J)), name = 'probinit')
print("done")

done


$$
% 概率递推
q_j\sum_{k\in J_{ij}^+} x_{ikj} p_{ikj}= p_{ijj'}, \forall i \in I, j \in J, j'\in J_{ij}^- \\
$$

In [13]:
m.addConstrs(((q[j] * gp.quicksum(w[i,k,j] for k in previous(i,j,J)) 
== p[i,j,j_p]) for i in I for j in J for j_p in later(i,j,bar_J)), name = 'probbalance')
print("done")

done


$$
% 尝试次数
\sum_{j\in J\cup\{i\}} \sum_{k\in J_{ij}^-} x_{ijk} \le R, \forall i\in I 
$$

In [14]:
m.addConstrs(((gp.quicksum(x[i,j,k] for j in J+[i] for k in later(i,j,bar_J))) <= max_visit_num for i in I), name = 'maxtry')
print("done")

done


$$

% 上界1
w_{ijk} \le p_{ijk}, \forall i \in I, j \in J+\{i\}, j'\in J_{ij}^- \\
% 上界2
w_{ijk} \le x_{ijk}, \forall i \in I, j \in J+\{i\}, j'\in J_{ij}^- \\
% 下界
w_{ijk} \ge p_{ijk} + x_{ijk} - 1, \forall i \in I, j \in J+\{i\}, j'\in J_{ij}^-\\
% 范围

$$

In [15]:
m.addConstrs((w[i,j,k] <= p[i,j,k] for i in I for j in J+[i] for k in later(i,j,bar_J)), name='u1b')
m.addConstrs((w[i,j,k] <= x[i,j,k] for i in I for j in J+[i] for k in later(i,j,bar_J)), name='u2b')
m.addConstrs((w[i,j,k] >= p[i,j,k] + x[i,j,k] -1 for i in I for j in J+[i] for k in later(i,j,bar_J)), name='lb')
print("done")

done


In [16]:
# 测试代码
for j in J:
    if j in [1,3,5,7,22,30]:
    # if j in [ 2,     6,     7,     8,     9,    10,    11,    12,    13,    15]:
        m.addConstr(y[j] == 1)
    else:
        m.addConstr(y[j] == 0)


## 求解

In [17]:
# m.write('test.lp')

In [18]:
m.Params.MIPGap = 0.000003
m.Params.timeLimit = 1000

# m.Params.LogFile =  "SolvingLog.log"
# m.write('Model.lp')

m.optimize()

# m.computeIIS()
# m.write('Model.ilp')
# m.write('Model.lp')

# m.write('Solution.sol')

print('求解完成')

Set parameter MIPGap to value 3e-06
Set parameter TimeLimit to value 1000
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 14970 rows, 10860 columns and 94245 nonzeros
Model fingerprint: 0x2b6dd2e5
Variable types: 7230 continuous, 3630 integer (3630 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+00]
  Objective range  [7e+03, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 1.677596e+07
Presolve removed 14915 rows and 10819 columns
Presolve time: 0.04s
Presolved: 55 rows, 41 columns, 206 nonzeros
Found heuristic solution: objective 1190140.5299
Variable types: 20 continuous, 21 integer (21 binary)

Root relaxation: objective 7.099467e+05, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent  

In [19]:
plan = get_sol(x, I, bar_J, max_visit_num)

In [24]:
print('%.2f\t%.2f\t%.2f\n' %(m.objVal, m.ObjBound, m.RunTime))

710075.82	710075.82	0.09

